In [1]:
import os
import numpy as np
import scipy.stats as st
import pandas as pd
import itertools
import scikit_posthocs

import iqplot

import bokeh.io
import bokeh.plotting
import bokeh.layouts
bokeh.io.output_notebook()

Loading BokehJS ...

## Exploratory Data Analysis

1. Uploading the whole excel file to read from all the sheets later.

In [3]:
path = os.path.join('..', 'data', 'benCom-long.xlsx')
data = pd.ExcelFile(path)

2. Creating a list of all sheets (tests) that we want to analyse.

In [4]:
tests = [i for i in data.sheet_names if ('Cohort' not in i)]
tests

['Beam',
 'Beam_steps',
 'Pole',
 'Wirehang',
 'Sticker',
 'Hindlimb',
 'Fecal_output',
 'Fecal_score',
 'Water_content',
 'Carmine_red',
 'Bead_exp']

3. Parsing the Excel file into separate datasets (1 test = 1 dataset) and storing them in a dictionary with keys = names of the tests/sheets

In [5]:
data_dict = {}

for test in tests:
    temp_df = data.parse(test)   

    trial_cols = []
    for i in list(temp_df.columns):
        if 'Trial' in (i):
            trial_cols.append(i)    
            
    temp_df = temp_df.melt(id_vars=['Cohort', 'Genotype', 'Microbiome', 'Cage', 'ID'], 
            value_vars=trial_cols, 
            var_name='Trial', 
            value_name='Measurement')
    
    if test == 'Beam_steps':
        temp_df.loc[:, 'Type'] = temp_df.loc[:, 'Trial'].str.split('_', expand=True)[0]
        temp_df.loc[:, 'Trial'] = temp_df.loc[:, 'Trial'].str.split('_', expand=True)[1]    

    temp_df = temp_df.dropna()
    data_dict[test] = temp_df

4. Plotting the raw data from all the tests for the EDA.

In [6]:
plots = []

for test in tests:
    df = data_dict[test]
    if test == 'Fecal_output':
        df = df.loc[df['Trial'] == 'Trial15']
    else:
        df = df.loc[df['Trial'] == 'Trial1']
        
    p = iqplot.stripbox(
        df,
        q='Measurement',
        q_axis='y',
        cats=['Genotype', 'Microbiome'],
        tooltips=[
            ('Mouse', '@ID'),
            ('Cage', '@Cage')
        ],
        title=test,
        color_column='ID',
    )
    p.y_range = bokeh.models.Range1d(0.8*np.min(data_dict[test].loc[:, 'Measurement'])-1, 1.1*np.max(data_dict[test].loc[:, 'Measurement']))
    plots.append(p)

lt = bokeh.layouts.grid(plots, ncols=2)
bokeh.io.show(lt)

for n, i in enumerate(plots):
    bokeh.io.save(
        i,
        filename=('../figures/' + 'EDA_Bencom_' + tests[n] + '.html'),
        title='Bokeh plot',
        resources=bokeh.resources.CDN)


## Pole descent

In [7]:
trial = 'Trial1'

work_df = data_dict['Pole']
work_df = work_df.loc[work_df['Trial'] == trial]

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [8]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(6.396406670252844), pvalue=np.float64(0.040835505828134874))

In [9]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_pole_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.010695,0.313449,0.040836
ASO_SPF,0.010695,1.000000,0.104922,0.040836
ASO_bC,0.313449,0.104922,1.000000,0.040836


## Beam cross

In [10]:
trial = 'Trial1'

work_df = data_dict['Beam']
work_df = work_df.loc[work_df['Trial'] == trial]

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [11]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(19.539070931093626), pvalue=np.float64(5.7166901878233e-05))

In [12]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_beam_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.000016,0.000004,0.000057
ASO_SPF,0.000016,1.000000,0.817787,0.000057
ASO_bC,0.000004,0.817787,1.000000,0.000057


## Sticker removal

In [13]:
trial = 'Trial1'

work_df = data_dict['Sticker']
work_df = work_df.loc[work_df['Trial'] == trial]

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [14]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(13.944527996805817), pvalue=np.float64(0.000937527939525821))

In [15]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_sticker_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.000056,0.037984,0.000938
ASO_SPF,0.000056,1.000000,0.020141,0.000938
ASO_bC,0.037984,0.020141,1.000000,0.000938


## Wirehang

In [16]:
work_df = data_dict['Wirehang'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n


### NHST

In [17]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(20.615435162861054), pvalue=np.float64(3.3374528074606554e-05))

In [18]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_wirehang_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.000005,0.000001,0.000033
ASO_SPF,0.000005,1.000000,0.763306,0.000033
ASO_bC,0.000001,0.763306,1.000000,0.000033


## Hindlimb

In [19]:
work_df = data_dict['Hindlimb'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [20]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(12.939849801939362), pvalue=np.float64(0.0015493420826462657))

In [21]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_hindlimb_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.000218,0.516273,0.001549
ASO_SPF,0.000218,1.000000,0.001773,0.001549
ASO_bC,0.516273,0.001773,1.000000,0.001549


## Fecal output

In [22]:
work_df = data_dict['Fecal_output'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial15']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j) & (work_df['Trial'] == 'Trial15'), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n.astype('int32')

### NHST

In [23]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(3.136874538356031), pvalue=np.float64(0.20837055514468025))

In [24]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_fecoutput_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.120116,0.153503,0.208371
ASO_SPF,0.120116,1.000000,0.847247,0.208371
ASO_bC,0.153503,0.847247,1.000000,0.208371


## Fecal score

In [25]:
work_df = data_dict['Fecal_score'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [26]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(19.117927136007715), pvalue=np.float64(7.056589752860742e-05))

In [27]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_fecal_score_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000e+00,6.283127e-07,0.006583,0.000071
ASO_SPF,6.283127e-07,1.000000e+00,0.003045,0.000071
ASO_bC,6.582984e-03,3.045379e-03,1.000000,0.000071


## Bead expulsion

In [28]:
work_df = data_dict['Bead_exp'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n.astype('int32')

### NHST

In [29]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(14.764041721977692), pvalue=np.float64(0.0006223419477943169))

In [30]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_beadexp_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.000034,0.007023,0.000622
ASO_SPF,0.000034,1.000000,0.031326,0.000622
ASO_bC,0.007023,0.031326,1.000000,0.000622


## Beam steps/slips

In [31]:
work_df = data_dict['Beam_steps']
work_df = work_df.pivot(index=['ID', 'Trial', 'Cohort', 'Genotype', 'Microbiome', 'Cage'], columns='Type', values='Measurement').reset_index()
work_df['Total_steps'] = work_df['Steps'] * 4
work_df['Slip_probability'] = work_df['Slips']/work_df['Total_steps']
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Slip_probability'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [32]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(22.74821132624602), pvalue=np.float64(1.1489170486191648e-05))

In [33]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_beadexp_pvals_conover_test.csv")
res

,ASO_bC,ASO_SPF,WT_SPF,Kruskal-Wallis
ASO_bC,1.000000e+00,8.423726e-01,1.686335e-07,0.000011
ASO_SPF,8.423726e-01,1.000000e+00,7.822715e-07,0.000011
WT_SPF,1.686335e-07,7.822715e-07,1.000000e+00,0.000011


## Carmine red

In [34]:
work_df = data_dict['Carmine_red']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n.astype('int32')

### NHST

In [35]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(5.089413843386435), pvalue=np.float64(0.0784960541004698))

In [36]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_beadexp_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.064419,0.043736,0.078496
ASO_SPF,0.064419,1.000000,0.933000,0.078496
ASO_bC,0.043736,0.933000,1.000000,0.078496


## Water content

In [37]:
work_df = data_dict['Water_content'].copy()
work_df = work_df.loc[work_df['Trial'] == 'Trial1']

group_vals = {}

effect_1 = 'Genotype'
effect_2 = 'Microbiome'

effect1_lst = work_df[effect_1].unique()
effect2_lst = work_df[effect_2].unique()

for i in effect1_lst:
    for j in effect2_lst:
        name = i + '_' + j
        n = work_df.loc[(work_df[effect_1] == i) & (work_df[effect_2] == j), 'Measurement'].values
        if len(n) != 0:
            group_vals[name] = n

### NHST

In [38]:
res_kw = st.kruskal(*list(group_vals.values()))
res_kw

KruskalResult(statistic=np.float64(9.846132118046107), pvalue=np.float64(0.007276785554293323))

In [39]:
res = scikit_posthocs.posthoc_conover(list(group_vals.values()))
names = list(group_vals.keys())
res = res.rename(columns={1:names[0], 2:names[1], 3:names[2]}, index={1:names[0], 2:names[1], 3:names[2]})
res['Kruskal-Wallis'] = res_kw.pvalue
res.to_csv("../output/Bencom_watercont_pvals_conover_test.csv")
res

,WT_SPF,ASO_SPF,ASO_bC,Kruskal-Wallis
WT_SPF,1.000000,0.001870,0.013974,0.007277
ASO_SPF,0.001870,1.000000,0.386822,0.007277
ASO_bC,0.013974,0.386822,1.000000,0.007277
